In [47]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/

In [46]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 18)
pd.set_option('display.width', 790)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_eval_sym_freq_results = 'df_eval_sym_freq_results'
fp_df_eval_sym_freq_results_validate = 'df_eval_sym_freq_results_validate'
fp_df_eval_sym_freq_results_test = 'df_eval_sym_freq_results_test'

START: Concatenate and pickle Train, Validate and Test results

In [48]:
df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)
run_type = 'train'
name_table = 'tbl_train'

# df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_validate)
# run_type = 'validate'
# name_table = 'tbl_val'

# df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_test)
# run_type = 'test'
# name_table = 'tbl_test'

In [49]:
# pd.crosstab(df['days_lookbacks'], df['grp(CAGR/UI)_mean'])
tbl = df.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
        .agg({'grp(CAGR)_mean':          ['count', 'mean', 'std',],
              'grp(CAGR/UI)_mean':       ['mean', 'std',],
              'grp(CAGR/retnStd)_mean':  ['mean', 'std',],
              'SPY_CAGR':                ['mean', 'std',],
              'SPY_CAGR/UI':             ['mean', 'std',],
              'SPY_CAGR/retnStd':        ['mean', 'std',],                           
              })
# tbl

In [50]:
tbl['dif_CAGR_mean'] = tbl['grp(CAGR)_mean', 'mean'] - tbl['SPY_CAGR', 'mean']
tbl['dif_CAGR/UI_mean'] = tbl['grp(CAGR/UI)_mean', 'mean'] - tbl['SPY_CAGR/UI', 'mean'] 
tbl['dif_CAGR/retnStd_mean'] = tbl['grp(CAGR/retnStd)_mean', 'mean'] - tbl['SPY_CAGR/retnStd', 'mean'] 
tbl['grp_CAGR/UI_mean/std'] = tbl['grp(CAGR/UI)_mean', 'mean'] / tbl['grp(CAGR/UI)_mean', 'std']  
tbl['SPY_CAGR/UI_mean/std'] = tbl['SPY_CAGR/UI', 'mean'] / tbl['SPY_CAGR/UI', 'std'] 
tbl['dif_CAGR/UI_mean/std'] = tbl['grp_CAGR/UI_mean/std']  - tbl['SPY_CAGR/UI_mean/std'] 
# tbl.sort_values(by='dif_CAGR/UI_mean', ascending=False, inplace=True)
# tbl.sort_values(by='dif_CAGR/UI_mean/std', ascending=False, inplace=True)
# tbl

In [51]:
tbl.sort_values(by='dif_CAGR/UI_mean/std', ascending=False, inplace=True)
tbl['pct_rank_d_CAGR/UI_m/s'] = tbl['dif_CAGR/UI_mean/std'].rank(pct=True)
tbl.head()
# len(tbl)

grp(CAGR)_mean                              grp(CAGR/UI)_mean               grp(CAGR/retnStd)_mean                SPY_CAGR              SPY_CAGR/UI               SPY_CAGR/retnStd              dif_CAGR_mean dif_CAGR/UI_mean dif_CAGR/retnStd_mean grp_CAGR/UI_mean/std SPY_CAGR/UI_mean/std dif_CAGR/UI_mean/std pct_rank_d_CAGR/UI_m/s
                                                  count          mean            std              mean           std                   mean           std      mean        std          mean           std             mean          std                                                                                                                                           
days_lookbacks    days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                                           
[30, 60, 120]     5         9                        19     57.429862      67.897798      5.472164e+07  1.141317e+08        3306.306624      3.792917e+03  4.447059   7.374901  7.806509e+03  2.785653e+04       397.709824   696.413640     52.982803     5.471383e+07        2908.596800              0.479460             0.280240             0.199220             1.000000    
[15, 30, 60, 120] 6         11                       11     39.455204      86.920259      7.793831e+03  1.595171e+04        1595.433261      3.448399e+03  2.426677   8.261264  3.374613e+03  1.106838e+04       193.795010   633.064882     37.028527     4.419218e+03        1401.638251              0.488589             0.304888             0.183701             0.991667    
                  5         9                        27    152.891418     481.513325      3.062780e+07  8.322446e+07        4682.348807      1.026730e+04  5.054576  13.506347  5.561986e+06  2.889536e+07       689.530407  2482.074996    147.836842     2.506581e+07        3992.818401              0.368014             0.192487             0.175527             0.983333    
                  4         3                        26  23442.979622  117762.828836      6.387102e+08  2.030204e+09      448219.696250      2.208939e+06  0.231208   3.078991  6.471672e+02  3.187209e+03        -1.501661   294.775017  23442.748415     6.387096e+08      448221.197911              0.314604             0.203051             0.111553             0.975000    
                  5         3                        33     15.391409      69.366656      1.479001e+06  5.210391e+06         653.825871      2.051227e+03 -0.103618   0.787082  7.780867e+04  4.467916e+05        -6.310224   122.250137     15.495027     1.401192e+06         660.136095              0.283856             0.174150             0.109706             0.966667

In [52]:
tbl['run_type'] = run_type

In [53]:
pickle_dump(tbl, path_data_dump, name_table)
# pickle_dump(tbl, path_data_dump, 'tbl.validate')
# pickle_dump(tbl, path_data_dump, 'tbl.test')

End: Concatenate and pickle Train, Validate and Test results

In [64]:
tbl_train = pickle_load(path_data_dump, 'tbl.train')
# tbl_train.head()
tbl_val = pickle_load(path_data_dump, 'tbl.validate')
# tbl_val.head()
tbl_test = pickle_load(path_data_dump, 'tbl.test')
# tbl_test.head()


In [65]:
print(f'len(tbl_train): {len(tbl_train)}')
print(f'len(tbl_val): {len(tbl_val)}')
print(f'len(tbl_test): {len(tbl_test)}')

len(tbl_train): 120
len(tbl_val): 117
len(tbl_test): 120


In [66]:
tbl_concat = pd.concat([tbl_train, tbl_val, tbl_test])
tbl_concat.sort_values(by=["days_lookbacks", "days_eval", "sym_freq_cnt"])
tbl_concat.head()

grp(CAGR)_mean                             grp(CAGR/UI)_mean               grp(CAGR/retnStd)_mean                   SPY_CAGR                 SPY_CAGR/UI               SPY_CAGR/retnStd               dif_CAGR_mean dif_CAGR/UI_mean dif_CAGR/retnStd_mean grp_CAGR/UI_mean/std SPY_CAGR/UI_mean/std dif_CAGR/UI_mean/std pct_rank_d_CAGR/UI_m/s run_type
                                                  count          mean           std              mean           std                   mean           std         mean           std          mean           std             mean           std                                                                                                                                                    
days_lookbacks    days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                                                          
[15, 30, 60, 120] 4         9                       231  3.019027e+07  2.185062e+08      3.921039e+13  2.988932e+14       4.181215e+08      2.607273e+09  2607.005710  39310.500243  5.210077e+09  7.862126e+10     84429.356152  1.268486e+06  3.018766e+07     3.920518e+13       4.180371e+08              0.131185             0.066268             0.064917             1.000000        train
[15, 30, 60]      4         9                       139  3.900034e+04  2.064576e+05      7.477254e+10  4.120590e+11       6.465760e+06      4.988826e+07  8598.489940  71407.917197  1.719477e+10  1.428161e+11    277647.576742  2.304214e+06  3.040185e+04     5.757777e+10       6.188112e+06              0.181461             0.120398             0.061063             0.991667        train
[30, 60, 120]     4         9                       128  1.405714e+05  9.797210e+05      2.674217e+11  1.958404e+12       8.855914e+06      6.661397e+07  4701.937810  52807.975285  9.401742e+09  1.056161e+11    152256.166211  1.704007e+06  1.358694e+05     2.580199e+11       8.703658e+06              0.136551             0.089018             0.047533             0.983333        train
                            6                       365  2.752167e+09  4.229883e+10      2.280436e+14  2.417940e+15       2.652122e+10      4.023273e+11  1650.419329  31273.269372  3.299372e+09  6.254662e+10     53943.101151  1.009131e+06  2.752165e+09     2.280403e+14       2.652117e+10              0.094313             0.052751             0.041563             0.975000        train
[15, 30, 60, 120] 4         5                       397  6.854585e+26  9.645220e+27      6.192848e+28  8.615225e+29       1.464170e+27      2.060262e+28  1517.577527  29986.454471  3.032301e+09  5.997305e+10     49326.115525  9.676117e+05  6.854585e+26     6.192848e+28       1.464170e+27              0.071883             0.050561             0.021322             0.966667        train

In [71]:
tbl_concat.groupby(level=[0,1,2]).sum().sort_values('pct_rank_d_CAGR/UI_m/s', ascending=False)

C:\Users\ping\AppData\Local\Temp\ipykernel_4216\3566629057.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tbl_concat.groupby(level=[0,1,2]).sum().sort_values('pct_rank_d_CAGR/UI_m/s', ascending=False)


grp(CAGR)_mean                             grp(CAGR/UI)_mean               grp(CAGR/retnStd)_mean                   SPY_CAGR                 SPY_CAGR/UI               SPY_CAGR/retnStd               dif_CAGR_mean dif_CAGR/UI_mean dif_CAGR/retnStd_mean grp_CAGR/UI_mean/std SPY_CAGR/UI_mean/std dif_CAGR/UI_mean/std pct_rank_d_CAGR/UI_m/s
                                                  count          mean           std              mean           std                   mean           std         mean           std          mean           std             mean           std                                                                                                                                           
days_lookbacks    days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                                                 
[30, 60, 120]     4         8                       287  2.251341e+31  3.495021e+32      1.016651e+32  1.578266e+33       2.981200e+31      4.599171e+32  2506.100201  38499.178890  5.003596e+09  7.699365e+10     82666.934367  1.244125e+06  2.251341e+31     1.016651e+32       2.981200e+31              1.460210             0.712894             0.747316             2.674145    
[15, 30, 60, 120] 5         7                       454  2.210351e+54  3.966332e+55      4.942494e+60  8.868989e+61       5.038295e+54      9.040897e+55    19.609688    205.238892  1.312977e+07  1.872187e+08      1086.509738  7.030503e+03  2.210351e+54     4.942494e+60       5.038295e+54              1.109533             0.546658             0.562875             2.598932    
[30, 60, 120]     4         9                       159  1.406140e+05  9.798341e+05      2.674723e+11  1.958628e+12       8.857921e+06      6.661959e+07  4711.536608  52823.851676  9.405926e+09  1.056353e+11    153574.866921  1.706507e+06  1.359025e+05     2.580663e+11       8.704346e+06              1.524426             1.352416             0.172009             2.545726    
                            4                       561  3.010245e+28  4.251791e+29      2.688766e+30  3.797721e+31       6.552447e+28      9.254940e+29  1511.058721  29887.662537  3.016252e+09  5.977174e+10     50506.064853  9.683658e+05  3.010245e+28     2.688766e+30       6.552447e+28              0.799411             0.495010             0.304401             2.447436    
                  6         9                       168  1.573623e+04  1.665716e+05      5.524852e+08  6.108887e+09       2.041877e+05      1.964075e+06     7.046354     16.209956  2.477272e+05  2.547085e+06       893.586793  1.053718e+03  1.572918e+04     5.522375e+08       2.032941e+05              2.647089             2.278452             0.368637             2.414530    
[15, 30, 60, 120] 4         5                       568  6.854585e+26  9.645220e+27      6.192848e+28  8.615225e+29       1.464170e+27      2.060262e+28  1525.013913  30009.444253  3.042862e+09  6.001776e+10     50928.917784  9.739818e+05  6.854585e+26     6.192848e+28       1.464170e+27              0.560918             0.503351             0.057567             2.368803    
[30, 60, 120]     5         7                       345  5.527698e+10  6.088126e+11      7.209697e+12  7.138298e+13       2.494028e+11      2.741740e+12     5.955643     16.701344  1.092793e+06  1.175840e+07       669.995134  2.126812e+03  5.527698e+10     7.209696e+12       2.494028e+11              1.372698             0.696621             0.676077             2.299359    
                  4         7                       333  5.028244e+09  7.749231e+10      1.005648e+16  1.549846e+17       3.949252e+10      6.052743e+11  2534.822299  38744.467980  5.064882e+09  7.748648e+10     83737.799857  1.255225e+06  5.028241e+09     1.00

In [57]:
# tbl_concat.head()
# _tbl = tbl_concat.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
#         .agg({'Rank_d_CAGR/UI_m/s':          ['count', 'mean', 'std',]
                       
#               })

In [59]:
# _tbl = tbl_concat.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
#         .agg({'Rank_d_CAGR/UI_m/s':          ['mean']                   
#               })

In [60]:
# # TODO not sorting correctly, should have three 1 on top 3 rows
# # https://stackoverflow.com/questions/28371308/sort-by-column-within-multi-index-level-in-pandas
# _tbt_concat.sort_values(by=["Rank_d_CAGR/UI_m/s", "dif_CAGR/UI_mean/std"], ascending=False, inplace=True)
# tbt_rank_sort =  _tbt_concat.head(300).copy()
# # _tbt_concat.sort_values(by=["days_lookbacks", "days_eval", "sym_freq_cnt"], ascending=True, inplace=True)
# # _tbt_concat.tail(400)

In [61]:
# tbt_rank_sort.head()

In [62]:
# from pandas import IndexSlice as idx
# rows = idx[:]
# # cols = idx['grp(CAGR)_mean']
# # cols = idx[['dif_CAGR_mean', 'dif_CAGR/UI_mean']]
# # cols = idx['grp(CAGR)_mean' , ['count','mean']]
# cols = idx[['dif_CAGR_mean', 'dif_CAGR/UI_mean', 'dif_CAGR/retnStd_mean', 'dif_CAGR/UI_mean/std', 'Rank_d_CAGR/UI_m/s', 'run_type']]
# # cols = idx['dif_CAGR_mean', [], 'dif_CAGR/UI_mean', []]
# tbt_rank_sort.loc[rows, cols].head(21)

In [63]:
# _tbt_concat.sort_values(by='dif_CAGR/UI_mean/std', ascending=False, inplace=True)